### Midterm Project

In [61]:
import os
import numpy
import pandas as pd
import pymongo
import json
from sqlalchemy import create_engine
from sqlalchemy.types import BLOB

#### Connect to SQL

In [62]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id="root"
pwd = "Passw0rd123"

src_dbname = "sakila"
dst_dbname = "sakila_dim_mart"

In [63]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}?charset=utf8mb4"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}?charset=utf8mb4"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace', dtype={"picture":BLOB()})
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append', use_unicode=True)
    
    connection.close()

In [64]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}?charset=utf8mb4"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

#sqlEngine.execute(f"ALTER DATABASE {dst_dbname} CHARACTER SET = utf8mb4 COLLATE = utf8mb4_unicode_ci;")

##### Create Dim Mart for Sakila Rentals:
1. Get relevant tables from Sakila database in SQL
* payment as part of fact table
* staff as a dimension table
* customer as a dimension table
* store as a dimension table
* film as a dimension table

In [65]:
sql_payment = "SELECT * FROM sakila.payment;"
df_payment = get_dataframe(user_id, pwd, host_name, src_dbname, sql_payment)
df_payment.rename(columns={"last_update":"payment_last_update"}, inplace=True)
df_payment.head(2)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date,payment_last_update
0,1,1,1,76,2.99,2005-05-25 11:30:37,2006-02-15 22:12:30
1,2,1,1,573,0.99,2005-05-28 10:35:23,2006-02-15 22:12:30


In [66]:
sql_customer = "SELECT * FROM sakila.customer;"
df_customer = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customer)
df_customer.rename(columns={"customer_id":"customer_key"}, inplace=True)
df_customer.head(2)

,customer_key,store_id,first_name,last_name,email,address_id,active,create_date,last_update
0,1,1,MARY,SMITH,MARY.SMITH@sakilacustomer.org,5,1,2006-02-14 22:04:36,2006-02-15 04:57:20
1,2,1,PATRICIA,JOHNSON,PATRICIA.JOHNSON@sakilacustomer.org,6,1,2006-02-14 22:04:36,2006-02-15 04:57:20


In [67]:
sql_staff = "SELECT * FROM sakila.staff;"
df_staff = get_dataframe(user_id, pwd, host_name, src_dbname, sql_staff)
df_staff.rename(columns={"staff_id":"staff_key"}, inplace=True)
df_staff.head(2)

,staff_key,first_name,last_name,address_id,picture,email,store_id,active,username,password,last_update
0,1,Mike,Hillyer,3,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,Mike.Hillyer@sakilastaff.com,1,1,Mike,8cb2237d0679ca88db6464eac60da96345513964,2006-02-15 03:57:16
1,2,Jon,Stephens,4,None,Jon.Stephens@sakilastaff.com,2,1,Jon,None,2006-02-15 03:57:16


In [68]:
sql_store = "SELECT * FROM sakila.store;"
df_store = get_dataframe(user_id, pwd, host_name, src_dbname, sql_store)
df_store.rename(columns={"store_id":"store_key"}, inplace=True)
df_store.head(2)

,store_key,manager_staff_id,address_id,last_update
0,1,1,1,2006-02-15 04:57:12
1,2,2,2,2006-02-15 04:57:12


In [69]:
sql_film = "SELECT * FROM sakila.film;"
df_film = get_dataframe(user_id, pwd, host_name, src_dbname, sql_film)
df_film.rename(columns={"film_id":"film_key"}, inplace=True)
df_film.drop(columns=["language_id", "original_language_id"], inplace=True)
df_film.head(2)

,film_key,title,description,release_year,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42


2. Get data from MongoDB

Populate MongoDB with JSON file of staff data from Sakila

In [70]:
mysql_uid = "root"
mysql_pwd = "Passw0rd123"

#atlas_cluster_name = "sandbox.zibbf"
#atlas_user_name = "m001-student"
#atlas_password = "m001-mongodb-basics"

conn_str = {"local" : f"mongodb://localhost:27017/",
#    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.mongodb.net"
}

print(f"Local Connection String: {conn_str['local']}")
#print(f"Atlas Connection String: {conn_str['atlas']}")

Local Connection String: mongodb://localhost:27017/


In [71]:
def get_mongo_dataframe(connect_str, db_name, collection, query):
    '''Create a connection to MongoDB'''
    client = pymongo.MongoClient(connect_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    return dframe

In [72]:
client = pymongo.MongoClient(conn_str["local"])
db = client[src_dbname]

# Gets the path of the Current Working Directory for this Notebook, and then Appends the 'data' directory.
data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"inventory" : 'sakila_inventory.json'}

for file in json_files:
    db.drop_collection(file)
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db[file]
        result = file.insert_many(json_object)
        
client.close()

In [73]:
query = {}
collection = "inventory"

df_inventory = get_mongo_dataframe(conn_str['local'], src_dbname, collection, query)
df_inventory.rename(columns={"last_update":"inventory_last_update"}, inplace=True)
df_inventory.head(2)

,inventory_id,film_id,store_id,inventory_last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17


3. Get data from file system

In [74]:
df_rental = pd.read_csv(os.path.join(data_dir, "sakila_rental.csv"), sep=';')
df_rental.rename(columns={"last_update":"rental_last_update"}, inplace=True)
df_rental.head(2)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,rental_last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53


### Join Payment, Rental, and Inventory Tables to Create Fact Table

In [75]:
df_rental_inv = pd.merge(df_rental, df_inventory, on="inventory_id", how="left")
df_rental_inv.drop(columns=["inventory_id"], inplace=True)
print(df_rental_inv.shape)
df_rental_inv.head(2)

(16044, 9)


,rental_id,rental_date,customer_id,return_date,staff_id,rental_last_update,film_id,store_id,inventory_last_update
0,1,2005-05-24 22:53:30,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,80,1,2006-02-15 05:09:17
1,2,2005-05-24 22:54:33,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,333,2,2006-02-15 05:09:17


In [76]:
# we join on customer_id and staff_id as wlel because each rental_id corresponds to one customer/staff and so does each payment_id
# the staff and customer should match for matching rentals and payments
df_rental_inv_pay = pd.merge(df_rental_inv, df_payment, on=["rental_id", "customer_id", "staff_id"], how="left")
df_rental_inv_pay.drop(columns=["rental_id", "payment_id"], inplace=True)
print(df_rental_inv_pay.shape)
df_rental_inv_pay.head(2)

(16044, 11)


,rental_date,customer_id,return_date,staff_id,rental_last_update,film_id,store_id,inventory_last_update,amount,payment_date,payment_last_update
0,2005-05-24 22:53:30,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,80,1,2006-02-15 05:09:17,2.99,2005-05-24 22:53:30,2006-02-15 22:13:16
1,2005-05-24 22:54:33,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,333,2,2006-02-15 05:09:17,NaN,NaT,NaT


In [77]:
ordered_columns = ["customer_id", "staff_id", "film_id", 
                   "store_id", "amount", "rental_date", "payment_date", "return_date", "rental_last_update", "inventory_last_update",
                  "payment_last_update"]

df_fctrentals = df_rental_inv_pay[ordered_columns]
df_fctrentals.rename(columns={"customer_id":"customer_key",
                              "staff_id":"staff_key",
                              "film_id": "film_key",
                             "store_id":"store_key"}, inplace=True)
df_fctrentals.insert(0, "fact_rental_key", range(1, df_fctrentals.shape[0]+1))

print(df_fctrentals.shape)

(16044, 12)


In [78]:
df_fctrentals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16044 entries, 0 to 16043
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   fact_rental_key        16044 non-null  int64         
 1   customer_key           16044 non-null  int64         
 2   staff_key              16044 non-null  int64         
 3   film_key               16044 non-null  int64         
 4   store_key              16044 non-null  int64         
 5   amount                 7966 non-null   float64       
 6   rental_date            16044 non-null  object        
 7   payment_date           7966 non-null   datetime64[ns]
 8   return_date            15861 non-null  object        
 9   rental_last_update     16044 non-null  object        
 10  inventory_last_update  16044 non-null  object        
 11  payment_last_update    7966 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(4)
memory

In [79]:
df_fctrentals.head(2)

,fact_rental_key,customer_key,staff_key,film_key,store_key,amount,rental_date,payment_date,return_date,rental_last_update,inventory_last_update,payment_last_update
0,1,130,1,80,1,2.99,2005-05-24 22:53:30,2005-05-24 22:53:30,2005-05-26 22:04:30,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:13:16
1,2,459,1,333,2,NaN,2005-05-24 22:54:33,NaT,2005-05-28 19:40:33,2006-02-15 21:30:53,2006-02-15 05:09:17,NaT


#### Write back fact table to SQL database

In [80]:
db_operation = "insert"
set_dataframe(user_id, pwd, host_name, dst_dbname, df_fctrentals, "fact_rentals", "fact_rental_key", db_operation)

In [81]:
date_columns = ["rental_date", "payment_date", "return_date", "rental_last_update", "inventory_last_update", "payment_last_update"]
sql_check = f"""SELECT * FROM {dst_dbname}.fact_rentals 
                  ORDER BY amount DESC"""
df_check = get_dataframe(user_id, pwd, host_name, src_dbname, sql_check)
print(df_check.shape)
df_check.head(15)

(16044, 12)


,fact_rental_key,customer_key,staff_key,film_key,store_key,amount,rental_date,payment_date,return_date,rental_last_update,inventory_last_update,payment_last_update
0,16035,195,2,771,2,11.99,2005-08-23 22:19:33,2005-08-23 22:19:33,2005-09-02 02:19:33,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:14:00
1,14759,116,2,324,1,11.99,2005-08-21 23:34:00,2005-08-21 23:34:00,2005-08-31 03:58:00,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:13:08
2,4381,591,2,580,2,11.99,2005-07-07 20:45:51,2005-07-07 20:45:51,2005-07-17 00:52:51,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:23:18
3,106,196,2,791,2,11.99,2005-05-25 18:18:19,2005-05-25 18:18:19,2005-06-04 00:01:19,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:14:00
4,135,304,1,21,1,10.99,2005-05-25 21:58:58,2005-05-25 21:58:58,2005-06-03 17:50:58,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:15:46
5,163,104,1,501,1,10.99,2005-05-26 02:26:23,2005-05-26 02:26:23,2005-06-04 06:36:23,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:13:02
6,690,433,1,879,1,10.99,2005-05-29 01:01:26,2005-05-29 01:01:26,2005-06-07 06:21:26,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:18:37
7,859,276,1,126,2,10.99,2005-05-30 02:45:16,2005-05-30 02:45:16,2005-06-08 07:28:16,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:15:15
8,14030,200,1,764,1,10.99,2005-08-20 21:31:52,2005-08-20 21:31:52,2005-08-29 00:13:52,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:14:04
9,14269,388,1,579,1,10.99,2005-08-21 06:26:48,2005-08-21 06:26:48,2005-08-30 10:34:48,2006-02-15 21:30:53,2006-02-15 05:09:17,2006-02-15 22:17:33


### Adding Dimension Tables

Find range of dates in fact table and convert to date_key (by running the SQL script)

In [82]:
for col in date_columns:
    df_fctrentals[col] = pd.to_datetime(df_fctrentals[col])

max_date = max(df_fctrentals[col].max() for col in date_columns)
min_date = min(df_fctrentals[col].min() for col in date_columns)

print("Earliest Date:", min_date)
print("Latest Date:", max_date)

Earliest Date: 2005-05-24 22:53:30
Latest Date: 2006-02-23 04:12:08


Insert dimension tables into SQL database

In [83]:
db_operation = "insert"

tables = [('dim_customer', df_customer, 'customer_key'),
          ('dim_staff', df_staff, 'staff_key'),
          ('dim_film', df_film, 'film_key'),
          ('dim_store', df_store, 'store_key')]

In [85]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

__Please finish creating the dimension tables by running the sakila_dim_mart.sql script to create dim_dates before proceeding to the next section__

### SQL Queries

In [93]:
# print some film information regarding the rentals that take place over the weeken (friday to sunday)
sql_query = f"""SELECT f.film_key, f.title, COUNT(*) AS `number of rentals`, AVG(fct.amount) AS `average price`
                FROM {dst_dbname}.fact_rentals fct
                INNER JOIN {dst_dbname}.dim_date d on fct.rental_date_key = d.date_key
                INNER JOIN {dst_dbname}.dim_film f ON fct.film_key = f.film_key
                WHERE d.day_of_week >= 5
                GROUP BY f.film_key, f.title
                ORDER BY `average price` DESC;"""

try:
    df_query = get_dataframe(user_id, pwd, host_name, dst_dbname, sql_query)
except:
    print(f"An error occurred in executing the query. Please check that the sakila_dim_mart.sql script was correctly executed\
    on {dst_dbname} to create the dim_date table.")

df_query.head(15)

,film_key,title,number of rentals,average price
0,48,BACKLASH UNDEFEATED,7,9.990
1,210,DARKO DORADO,6,9.990
2,946,VIRTUAL SPOILERS,4,9.990
3,710,RAGE GAMES,6,9.990
4,580,MINE TITANS,7,9.990
5,662,PATHS CONTROL,2,9.990
6,390,GUYS FALCON,3,9.990
7,65,BEHAVIOR RUNAWAY,7,9.490
8,501,KISSING DOLLS,6,9.490
9,415,HIGH ENCINO,10,8.990
